Diciembre 2024

Autor: Alejandra Cruz Vargas

[Data](https://www.kaggle.com/datasets/nickcantalupa/nfl-team-data-2003-2023)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline

# Cargar los datos
data = pd.read_csv('/content/NFL_team_stats_2003_2023.csv')

# Manejo de valores nulos
data['mov'] = data['mov'].fillna(data['mov'].median())  # Rellenar los valores nulos con la mediana
data['ties'] = data['ties'].fillna(data['ties'].mode()[0])  # Rellenar con la moda

# Codificación one-hot de la columna 'team'
data_one_hot = pd.get_dummies(data, columns=['team'], prefix='team')

# Dividir los datos en variables predictoras y variable objetivo
X = data_one_hot[['year', 'wins', 'losses','g',
       'team_Arizona Cardinals', 'team_Atlanta Falcons',
       'team_Baltimore Ravens', 'team_Buffalo Bills', 'team_Carolina Panthers',
       'team_Chicago Bears', 'team_Cincinnati Bengals',
       'team_Cleveland Browns', 'team_Dallas Cowboys', 'team_Denver Broncos',
       'team_Detroit Lions', 'team_Green Bay Packers', 'team_Houston Texans',
       'team_Indianapolis Colts', 'team_Jacksonville Jaguars',
       'team_Kansas City Chiefs', 'team_Las Vegas Raiders',
       'team_Los Angeles Chargers', 'team_Los Angeles Rams',
       'team_Miami Dolphins', 'team_Minnesota Vikings',
       'team_New England Patriots', 'team_New Orleans Saints',
       'team_New York Giants', 'team_New York Jets', 'team_Oakland Raiders',
       'team_Philadelphia Eagles', 'team_Pittsburgh Steelers',
       'team_San Diego Chargers', 'team_San Francisco 49ers',
       'team_Seattle Seahawks', 'team_St. Louis Rams',
       'team_Tampa Bay Buccaneers', 'team_Tennessee Titans',
       'team_Washington Commanders', 'team_Washington Football Team',
       'team_Washington Redskins']]
y = (data_one_hot['win_loss_perc'] > 0.5).astype(int)  # 1 si ganó, 0 si no

# Dividir los datos en entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE para balancear el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Normalización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Modelo de regresión logística
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Entrenamiento del modelo
log_reg.fit(X_train_scaled, y_train_resampled)

# Predicciones en conjunto de validación y prueba
y_val_pred = log_reg.predict(X_val_scaled)
y_test_pred = log_reg.predict(X_test_scaled)

# Evaluación
print("Evaluación en Conjunto de Validación:")
print(classification_report(y_val, y_val_pred))
print("Matriz de confusión en validación:")
print(confusion_matrix(y_val, y_val_pred))

print("\nEvaluación en Conjunto de Prueba:")
print(classification_report(y_test, y_test_pred))
print("Matriz de confusión en prueba:")
print(confusion_matrix(y_test, y_test_pred))

# ------------------ PREDICCIÓN Y COMPARACIÓN DE EQUIPOS ------------------

# Función para generar el vector de entrada para predicciones
def crear_vector(data_columns, equipo, year, wins, losses, g):
    vector = pd.DataFrame([{col: 0 for col in data_columns}])  # Crear un dataframe con ceros
    vector['year'] = year
    vector['wins'] = wins
    vector['losses'] = losses
    vector['g'] = g
    if f'team_{equipo}' in data_columns:  # Asignar 1 si el equipo está presente
        vector[f'team_{equipo}'] = 1
    return vector

# Función para obtener el historial de un equipo
def obtener_historial(data, equipo):
    columnas_equipo = [col for col in data.columns if equipo.lower() in col.lower()]
    if not columnas_equipo:
        raise ValueError(f"Equipo {equipo} no encontrado en las columnas de datos.")
    equipo_data = data[data[columnas_equipo[0]] == 1]
    historial = {
        'avg_mov': equipo_data['mov'].mean() if not equipo_data['mov'].isna().all() else 0,
        'total_wins': equipo_data['wins'].sum(),
        'recent_win_pct': equipo_data['win_loss_perc'].iloc[-5:].mean() if len(equipo_data) >= 5 else equipo_data['win_loss_perc'].mean()
    }
    return historial

# Crear vectores de entrada para los dos equipos
data_columns = X.columns
equipo_1 = 'Green Bay Packers'
equipo_2 = 'Seattle Seahawks'

vector_1 = crear_vector(data_columns, equipo_1, 2024, 9, 4, 13)
vector_2 = crear_vector(data_columns, equipo_2, 2024, 8, 5, 13)

# Obtener el historial de ambos equipos
historial_1 = obtener_historial(data_one_hot, equipo_1)
historial_2 = obtener_historial(data_one_hot, equipo_2)

# Imprimir los historiales de los equipos
print(f"\nHistorial de {equipo_1}: {historial_1}")
print(f"Historial de {equipo_2}: {historial_2}")

# Normalización de los vectores
vector_1_scaled = scaler.transform(vector_1)
vector_2_scaled = scaler.transform(vector_2)

# Predicciones de probabilidad
prob_equipo_1 = log_reg.predict_proba(vector_1_scaled)[0][1]
prob_equipo_2 = log_reg.predict_proba(vector_2_scaled)[0][1]

# Comparación final y determinación del ganador
print("\n---- RESULTADOS ----")
print(f"Probabilidad de ganar {equipo_1}: {prob_equipo_1:.4f}")
print(f"Probabilidad de ganar {equipo_2}: {prob_equipo_2:.4f}")

if prob_equipo_1 > prob_equipo_2:
    print(f"Predicción: ¡{equipo_1} ganará contra {equipo_2}!")
elif prob_equipo_1 < prob_equipo_2:
    print(f"Predicción: ¡{equipo_2} ganará contra {equipo_1}!")
else:
    print("Predicción: Es un empate según el modelo.")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Evaluación en Conjunto de Validación:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        51
           1       0.94      0.98      0.96        50

    accuracy                           0.96       101
   macro avg       0.96      0.96      0.96       101
weighted avg       0.96      0.96      0.96       101

Matriz de confusión en validación:
[[48  3]
 [ 1 49]]

Evaluación en Conjunto de Prueba:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        52
           1       0.98      1.00      0.99        49

    accuracy                           0.99       101
   macro avg       0.99      0.99      0.99       101
weighted avg       0.99      0.99      0.99       101

Matriz de confusión en prueba:
[[51  1]
 [ 0 49]]

Historial de Green Bay Packers: {'avg_mov': 2.1952380952380954, 'total_wins': 207, 'recent_win_pct': 0.6782}
Historial de Seattle Seahawks: {'avg_mov': 0.87619047619047